In [1]:
import os
from pathlib import Path
from collections import Counter

FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day8.txt'

with open(FOLDER / in_file) as f:
    data = [s.strip().split(' | ') for s in f]


In [2]:
from collections import Counter

'''
Each display digit expressed as an 8 bit int based 
on which segment is lit up
'''
bits = {
    0b1110111: '0',
    0b0010010: '1',
    0b1011101: '2',
    0b1011011: '3',
    0b0111010: '4',
    0b1101011: '5',
    0b1101111: '6',
    0b1010010: '7',
    0b1111111: '8',
    0b1111011: '9'
}

def rev_dict(l):
    '''Map character counts to lists of characters'''
    counts = Counter(''.join(l))
    res = {}
    for k, v in counts.items():
        res.setdefault(v, []).append(k)
    return res

def map_wires(l):
    '''
    Determine which input character lights up which segment.
    Return a dict mapping character to the bit representing
    segment.

    The info from two sets of letter counts is enough to decode
    all letter to wire mappings
    '''
    l = l.split()
    
    # letter counts for all tokens
    counts = rev_dict(l)
    # letter counts for tokens except '4'
    counts_no_4 = rev_dict([s for s in l if len(s) != 4])

    # segments b, d, e, f have unique counts in one of the dicts
    bits = {
        64: None,              #a
        32: counts[6][0],      #b
        16: None,              #c
        8:  counts_no_4[6][0], #d
        4:  counts[4][0],      #e
        2:  counts[9][0],      #f
        1:  None               #g
    }
    # deduce remaining three
    bits[64] = next(c for c in counts_no_4[8] if c != bits[2])  #a
    bits[16] = next(c for c in counts[8] if c != bits[64])      #c
    bits[1]  = next(c for c in counts_no_4[7] if c != bits[16]) #g 

    return {v:k for k, v in bits.items()}

def word_to_digit(word, wires):
    '''
    Given a 'word' from the cypher text, produce the digit it represents.
    `wires` is the mapping of bits to segments from map_wires.
    '''
    segment_total = sum(wires[c] for c in word)
    return bits[segment_total]

def decode_input(s, wires):
    '''
    For each token in s get the digit it represents. Assemble
    them and return the str -> int conversion
    '''
    tokens = s.split()
    s = ''.join([word_to_digit(t, wires) for t in tokens])
    return int(s)

In [3]:
def solution_one(data):
    count = 0
    for key, cypher in data:
        count += len([s for s in cypher.split() if len(s) in [2, 3, 4, 7]])
    
    return count

solution_one(data)

548

In [4]:
def solution_two(data):
    total = 0
    for key, cypher in data:
        wires = map_wires(key)
        total += decode_input(cypher, wires)
    return total

print(solution_two(data))


1074888


In [34]:
%timeit solution_two(data)

4.58 ms ± 26.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
